In [3]:
from datetime import datetime
import pandas as pd
import requests
from typing import *
import time

class BinanceClient:
    def __init__(self, futures=False):
        self.exchange = "BINANCE"
        self.futures = futures

        if self.futures:
            self._base_url = "https://fapi.binance.com"
        else:
            self._base_url = "https://api.binance.com"

        self.symbols = self._get_symbols()

    def _make_request(self, endpoint: str, query_parameters: Dict):
        try:
            response = requests.get(self._base_url + endpoint, params=query_parameters)
        except Exception as e:
            print("Connection error while making request to %s: %s", endpoint, e)
            return None

        if response.status_code == 200:
            return response.json()
        else:
            print("Error while making request to %s: %s (status code = %s)",
                         endpoint, response.json(), response.status_code)
            return None

    def _get_symbols(self) -> List[str]:

        params = dict()

        endpoint = "/fapi/v1/exchangeInfo" if self.futures else "/api/v3/exchangeInfo"
        data = self._make_request(endpoint, params)

        symbols = [x["symbol"] for x in data["symbols"]]

        return symbols

    def get_historical_data(self, symbol: str, interval: Optional[str] = "1m", start_time: Optional[int] = None, end_time: Optional[int] = None, limit: Optional[int] = 1500):

        params = dict()

        params["symbol"] = symbol
        params["interval"] = interval
        params["limit"] = limit

        if start_time is not None:
            params["startTime"] = start_time
        if end_time is not None:
            params["endTime"] = end_time

        endpoint = "/fapi/v1/klines" if self.futures else "/api/v3/klines"
        raw_candles = self._make_request(endpoint, params)

        candles = []

        if raw_candles is not None:
            for c in raw_candles:
                candles.append((float(c[0]), float(c[1]), float(c[2]), float(c[3]), float(c[4]), float(c[5]),))
            return candles
        else:
            return None

def ms_to_dt_utc(ms: int) -> datetime:
    return datetime.utcfromtimestamp(ms / 1000)

def ms_to_dt_local(ms: int) -> datetime:
    return datetime.fromtimestamp(ms / 1000)

def GetDataFrame(data):
    df = pd.DataFrame(data, columns=['Timestamp', "Open", "High", "Low", "Close", "Volume"])
    df["Timestamp"] = df["Timestamp"].apply(lambda x: ms_to_dt_local(x))
    df['Date'] = df["Timestamp"].dt.strftime("%d/%m/%Y")
    df['Time'] = df["Timestamp"].dt.strftime("%H:%M:%S")
    column_names = ["Date", "Time", "Open", "High", "Low", "Close", "Volume"]
    df = df.set_index('Timestamp')
    df = df.reindex(columns=column_names)

    return df

def GetHistoricalData(client, symbol, interval, start_time, end_time, limit=1500):
    collection = []

    while start_time < end_time:
        data = client.get_historical_data(symbol, interval, start_time=start_time, end_time=end_time, limit=limit)
        print(client.exchange + " " + symbol + " : Collected " + str(len(data)) + " initial data from "+ str(ms_to_dt_local(data[0][0])) +" to " + str(ms_to_dt_local(data[-1][0])))
        start_time = int(data[-1][0] + 1000)
        collection +=data
        time.sleep(1.1)

    return collection

In [4]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import calendar

def get_klines_iter(symbol, interval, start, end = None, limit=1000):
    # start and end must be isoformat YYYY-MM-DD
    # We are using utc time zone

    # the maximum records is 1000 per each Binance API call

    df = pd.DataFrame()

    if start is None:
        print('start time must not be None')
        return
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000

    if end is None:
        dt = datetime.now(timezone.utc)
        utc_time = dt.replace(tzinfo=timezone.utc)
        end = int(utc_time.timestamp()) * 1000
        return
    else:
        end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000
    last_time = None

    while len(df) == 0 or (last_time is not None and last_time < end):
        url = 'https://api.binance.com/api/v3/klines?symbol=' + \
              symbol + '&interval=' + interval + '&limit=1000'
        if(len(df) == 0):
            url += '&startTime=' + str(start)
        else:
            url += '&startTime=' + str(last_time)

        url += '&endTime=' + str(end)
        df2 = pd.read_json(url)
        df2.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']
        dftmp = pd.DataFrame()
        dftmp = pd.concat([df2, dftmp], axis=0, ignore_index=True, keys=None)

        dftmp.Opentime = pd.to_datetime(dftmp.Opentime, unit='ms')
        dftmp['Date'] = dftmp.Opentime.dt.strftime("%d/%m/%Y")
        dftmp['Time'] = dftmp.Opentime.dt.strftime("%H:%M:%S")
        dftmp = dftmp.drop(['Quote asset volume', 'Closetime', 'Opentime',
                      'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore'], axis=1)
        column_names = ["Date", "Time", "Open", "High", "Low", "Close", "Volume"]
        dftmp.reset_index(drop=True, inplace=True)
        dftmp = dftmp.reindex(columns=column_names)
        string_dt = str(dftmp['Date'][len(dftmp) - 1]) + 'T' + str(dftmp['Time'][len(dftmp) - 1]) + '.000Z'
        utc_last_time = datetime.strptime(string_dt, "%d/%m/%YT%H:%M:%S.%fZ")
        last_time = (utc_last_time - datetime(1970, 1, 1)) // timedelta(milliseconds=1)
        df = pd.concat([df, dftmp], axis=0, ignore_index=True, keys=None)
    return df

In [5]:
get_klines_iter('BTCUSDT', '4h', '2016-01-01', '2022-08-27').to_csv("BTC_4h.csv")
get_klines_iter('ETHUSDT', '4h', '2016-01-01', '2022-08-27').to_csv("ETH_4h.csv")
get_klines_iter('SOLUSDT', '4h', '2016-01-01', '2022-08-27').to_csv("SOL_4h.csv")
get_klines_iter('DOGEUSDT','4h', '2016-01-01', '2022-08-27').to_csv("DOGE_4h.csv")
get_klines_iter('BNBUSDT', '4h', '2016-01-01', '2022-08-27').to_csv("BNB_4h.csv")
get_klines_iter('ADAUSDT', '4h', '2016-01-01', '2022-08-27').to_csv("ADA_4h.csv")
get_klines_iter('XRPUSDT', '4h', '2016-01-01', '2022-08-27').to_csv("XRP_4h.csv")


In [4]:
get_klines_iter('DOGEUSDT','4h', '2016-01-01', '2022-08-27').to_csv("DOGE_4h.csv")


In [4]:
get_klines_iter('BTCUSDT', '8h', '2017-01-01', '2022-07-09').to_csv("BTC_8h.csv")
get_klines_iter('ETHUSDT', '8h', '2017-01-01', '2022-07-09').to_csv("ETH_8h.csv")
get_klines_iter('SOLUSDT', '8h', '2017-01-01', '2022-07-09').to_csv("SOL_8h.csv")
get_klines_iter('DOGEUSDT','8h', '2017-01-01','2022-07-09').to_csv("DOGD_8h.csv")

get_klines_iter('BNBUSDT', '8h', '2017-01-01', '2022-07-09').to_csv("BNB_8h.csv")
get_klines_iter('ADAUSDT', '8h', '2017-01-01', '2022-07-09').to_csv("ADA_8h.csv")
get_klines_iter('XRPUSDT', '8h', '2017-01-01', '2022-07-09').to_csv("XRP_8h.csv")


In [5]:
get_klines_iter('BTCUSDT', '12h', '2017-01-01', '2022-07-09').to_csv("BTC_12h.csv")
get_klines_iter('ETHUSDT', '12h', '2017-01-01', '2022-07-09').to_csv("ETH_12h.csv")
get_klines_iter('SOLUSDT', '12h', '2017-01-01', '2022-07-09').to_csv("SOL_12h.csv")
get_klines_iter('DOGEUSDT','12h', '2017-01-01', '2022-07-09').to_csv("DOGD_12h.csv")
get_klines_iter('BNBUSDT', '12h', '2017-01-01', '2022-07-09').to_csv("BNB_12h.csv")
get_klines_iter('ADAUSDT', '12h', '2017-01-01', '2022-07-09').to_csv("ADA_12h.csv")
get_klines_iter('XRPUSDT', '12h', '2017-01-01', '2022-07-09').to_csv("XRP_12h.csv")


In [40]:
get_klines_iter('ETHUSDT', '1d', '2017-01-01', '2022-06-21')

,Date,Time,Open,High,Low,Close,Volume
0,17/08/2017,00:00:00,301.13,312.18,298.00,302.00,7.030710e+03
1,18/08/2017,00:00:00,302.00,311.79,283.94,293.96,9.537846e+03
2,19/08/2017,00:00:00,293.31,299.90,278.00,290.91,2.146198e+03
3,20/08/2017,00:00:00,289.41,300.53,282.85,299.10,2.510139e+03
4,21/08/2017,00:00:00,299.10,346.52,294.60,323.29,5.219445e+03
...,...,...,...,...,...,...,...
1766,17/06/2022,00:00:00,1068.50,1118.00,1051.11,1086.94,1.220678e+06
1767,18/06/2022,00:00:00,1086.93,1097.58,881.56,995.13,2.308984e+06
1768,19/06/2022,00:00:00,995.12,1158.62,936.15,1128.53,2.018599e+06
1769,20/06/2022,00:00:00,1128.53,1170.00,1053.00,1128.24,1.721773e+06


In [6]:
get_klines_iter('BTCUSDT', '1d', '2017-01-01', '2022-07-09').to_csv("BTC_1d.csv")
get_klines_iter('ETHUSDT', '1d', '2017-01-01', '2022-07-09').to_csv("ETH_1d.csv")
get_klines_iter('SOLUSDT', '1d', '2017-01-01', '2022-07-09').to_csv("SOL_1d.csv")
get_klines_iter('DOGEUSDT','1d', '2017-01-01', '2022-07-09').to_csv("DOGD_1d.csv")
get_klines_iter('BNBUSDT', '1d', '2017-01-01', '2022-07-09').to_csv("BNB_1d.csv")
get_klines_iter('ADAUSDT', '1d', '2017-01-01', '2022-07-09').to_csv("ADA_1d.csv")
get_klines_iter('XRPUSDT', '1d', '2017-01-01', '2022-07-09').to_csv("XRP_1d.csv")
